# practice run

### load data

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import math
import random

# data
import datetime
import itertools
import json

# sklearn
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# custom class
from features_yj import Features

In [3]:
t = Features()
df = t.run_all()

#show id - 25 일단 중간중간 데이터 살펴볼 때 필요해서 넣음
xcol = [5,6,7,8,16,17,18,21,22,23,25,28,30,32,34,35,36]
xcol2 = list(range(38,65))
xcol = xcol + xcol2

# important! check feature names (make sure indexing is correct)
df.columns[xcol].tolist()

### NA imputation

In [24]:
lag_col = ['lag_scode_count',
 'lag_mcode_price',
 'lag_mcode_count',
 'lag_bigcat_price',
 'lag_bigcat_count',
 'lag_bigcat_price_day',
 'lag_bigcat_count_day',
 'lag_small_c_price',
 'lag_small_c_count']

for col in lag_col:
    df[col] = df[col].fillna(0)

### encode

In [84]:
df2 = df.iloc[:,xcol]
# change categorical variables to dummies
# hours_inweek 일단 numerical하게 생각, men도 수정 전이라 일단 뺌
cat_col = ['min_start','japp','parttime', 'primetime','exposed','상품군','weekdays', 'small_c','middle_c','big_c', 'pay']
num = df2.drop(columns = cat_col)
X1 = df2[cat_col]

# Onehotencoder
ohe = OneHotEncoder()
ohe.fit(X1)
onehotlabels = ohe.transform(X1).toarray()
cat_labels = ohe.get_feature_names(['min_start','japp','parttime', 'primetime','exposed','상품군','weekdays', 'small_c','middle_c','big_c', 'pay'])
cat = pd.DataFrame(onehotlabels, columns=cat_labels)

df3 = num.join(cat)

In [130]:
df3.shape #(35379, 432)

(35379, 432)

### divide into train/test

In [135]:
# 1~8월 / 9~12월
# 23813
df3_train = df3.iloc[:23813,:]
df3_test = df3.iloc[23813:,:]

numeric_colnum = df3_train.columns.get_indexer(['판매단가','취급액','vratings']).tolist()
feature_set = df3_train.iloc[:,numeric_colnum]

# identify outliers in the training dataset
iso = IsolationForest(n_estimators=50, max_samples=50, contamination=float(0.05),max_features=1.0)
iso.fit(feature_set)
pred = iso.predict(feature_set)
feature_set['anomaly']=pred
outliers=feature_set.loc[feature_set['anomaly']==-1]
outlier_index=list(outliers.index)
df3_train = df3_train.loc[~df3_train.index.isin(outlier_index)].reset_index()

### remove outliers

In [136]:
X_train = df3_train.drop(columns = '취급액')
X_test = df3_test.drop(columns = '취급액')
y_train = df3_train.iloc[:,1]
y_test = df3_test.iloc[:,1]

In [140]:
y_test.shape

(11566,)